In [ ]:
# Hello!
# In DRC they grow far more crops than elsewhere, so I want to separate the parts for cropdiv

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline
from scipy.stats import pearsonr
import statsmodels.api as sm

In [ ]:
def normalise(dataframe):
    for i in dataframe.columns:
        (m,s) = (dataframe[i].mean(), dataframe[i].std())
        dataframe.loc[:,i]=dataframe.loc[:,i]-m
        dataframe.loc[:,i]=np.divide(dataframe.loc[:,i],s)
        print ("for column ",str(i), ", the mean was ",m," and the standard deviation was ",s)
    return dataframe

In [ ]:
def norm_series(series):
    (m,s) = (series.mean(), series.std())
    series=series-m
    series=np.divide(series,s)
    return series

In [ ]:
indicators = pd.read_csv("3PCAonImputedValues_output",encoding = "ISO-8859-1")

In [ ]:
analysis = None
possible_analyses = ["PCA","PAF"]
analysis = input("PCA or PAF? ")
while (analysis in possible_analyses) == False:
    analysis = input("You have to choose PCA or PAF.\n PCA or PAF? ")

In [ ]:
# name the important columns, and normalise them so that the unit is standard deviations from the mean
if analysis == 'PCA':
    factors_temp = normalise(indicators[['Food Security PCA','Diet Quality PCA','Money PCA']])
    factors_temp.rename(columns={'Food Security PCA':'Food Security Factor','Diet Quality PCA':'Diet Quality Factor',
                                 'Money PCA':'Money Factor'}, inplace=True)
    
else:
    factors_temp = normalise(indicators[['Food Security PAF','Diet Quality PAF','Money PAF']])
    factors_temp.rename(columns={'Food Security PAF':'Food Security Factor','Diet Quality PAF':'Diet Quality Factor',
                                 'Money PAF':'Money Factor'}, inplace=True)

indicators = pd.concat([indicators, factors_temp],axis=1)


In [ ]:
factors=['Food Security Factor','Diet Quality Factor','Money Factor']

In [ ]:
# Get ready to group by village
indicators['Region'].fillna('Not recorded', inplace=True)
indicators['Village'].fillna('None', inplace=True)
indicators['Coun Reg Vill']=indicators['Country']+indicators['Region']+indicators['Village'].astype(str)
#indicators.head()

In [ ]:
def ols_params(x,y):
    testmod = sm.OLS(y,sm.add_constant(x))
    tes = testmod.fit()
    a = tes.params[1]
    s = tes.bse[1]
    rsq = tes.rsquared
    return (a,s, rsq)

In [ ]:
# get rid of the small number of missing values
indi=indicators[indicators['CropDiv']<10000]
indic_DRC = indi[indi['Country']=='DRC']
indic_rest = indi[indi['Country']!='DRC']

In [ ]:
# Look at crop diversity against the factors.
def factors_cropdiv_household(data, save=None, alpha=.03):
    fig = plt.figure(figsize = (10,10))
    ax = fig.subplots(3)
    corrs = ['CropDiv','Money Factor','Food Security Factor','Diet Quality Factor']
    ax[2].scatter(x=data['CropDiv'],y= data['Money Factor'], alpha = alpha)
    ax[2].set_ylabel('Money Factor')
    ax[0].scatter(x=data['CropDiv'],y= data['Diet Quality Factor'], alpha=alpha)
    ax[0].set_ylabel('Diet Quality Factor')
    ax[1].scatter(x=data['CropDiv'],y= data['Food Security Factor'], alpha=alpha)
    ax[1].set_ylabel('Food Security Factor')


    for i in range (3):
        ax[i].set_xlabel('Household crop diversity')
        r=pearsonr(data[corrs[0]],data[corrs[1+i]])
        (a,s, Rsq)=ols_params(data['CropDiv'], data[corrs[1+i]])
        fig.text(.6,.14+.265*i,'Corr coeff: '+str(round(r[0],3))+' Sig level: '+str(round(r[1],5))+' n = '+str(len(data))+
                  '\nReg coeff: '+str(round(a,3))+' Std err: '+str(round(s,3))+' Rsq: '+str(round(Rsq,3)))
    if save:
        if analysis == "PAF":
            fig.savefig(str('4b PAF '+save))
        else:
            fig.savefig(str('4b PCA '+save))

In [ ]:
factors_cropdiv_household(indic_DRC, alpha = .3, save='factors DRC by household')
factors_cropdiv_household(indic_rest, save='factors without DRC by household')

In [ ]:
first_grouping_DRC = indic_DRC[['CropDiv','Food Security Factor',
       'Diet Quality Factor', 'Money Factor', 'Coun Reg Vill']].groupby('Coun Reg Vill').mean()
first_grouping_rest = indic_rest[['CropDiv','Food Security Factor',
       'Diet Quality Factor', 'Money Factor', 'Coun Reg Vill']].groupby('Coun Reg Vill').mean()

In [ ]:
# Repeat the scatter graphs by village, not by household
# The correlation coefficients are now much larger, and the shapes can be seen on the scatter graphs
def factors_cropdiv_village(data, save=None, alpha=0.3):
    fig = plt.figure(figsize = (10,12))
    axes = fig.subplots(3, gridspec_kw={'hspace':.4})
    for i in range(3):
        axes[i].scatter(x=data['CropDiv'], y=data[data.columns[1+i]], alpha = alpha)
        axes[i].set_ylabel(str(data.columns[1+i]))
        axes[i].set_xlabel('Average crop diversity in village')
        rp = pearsonr(data['CropDiv'], data[data.columns[1+i]])
        r = round(rp[0], 3)
        p=round(rp[1],5)
        (a, s, Rsq) = ols_params(data['CropDiv'], data[data.columns[1+i]])
        fig.text(.58,.69-.28*i,'Corr coef: '+str(r)+' Sig level: '+str(p)+
                     ' (n = '+str(len(data))+')'+
                  '\nReg coeff: '+str(round(a,3))+' Std err: '+str(round(s,3))+' Rsq: '+str(round(Rsq,3)))
    if save:
        if analysis=='PCA':
            fig.savefig('4b PCA '+save)
        else:
            fig.savefig('4b PAF '+save)

In [ ]:
factors_cropdiv_village(first_grouping_DRC, alpha=.6, save = 'factors DRC by village')
factors_cropdiv_village(first_grouping_rest, save = 'factors without DRC by village')

In [ ]:
# next, I want to split it up by market orientation
# I had 4 market orientation quartiles before.
prep_DRC=indic_DRC[indic_DRC['FAMarketOrientation']<1.1]
prep_rest=indic_rest[indic_rest['FAMarketOrientation']<1.1]

second_grouping_DRC = prep_DRC[['CropDiv','Food Security Factor',
       'Diet Quality Factor', 'Money Factor', 'Coun Reg Vill', 'FAMarketOrientation']].groupby('Coun Reg Vill').mean()
second_grouping_rest = prep_rest[['CropDiv','Food Security Factor',
       'Diet Quality Factor', 'Money Factor', 'Coun Reg Vill', 'FAMarketOrientation']].groupby('Coun Reg Vill').mean()

In [ ]:
# make market orientation quartile columns
def quartile(data, column, name=None):
    if name:
        quart=name
    else:
        quart=str(column)+'quartile'
    data[quart]=0
    for j in range(4):
        q = data[column].quantile(0.25*(j+1))
        data[quart]=np.where(data[column]>q, data[quart]+1, data[quart])
    return data
    
#second_grouping_l['MAQuartile']=0
#for j in range(4):
 #   print(second_grouping[['FAMarketOrientation','MAQuartile']][11:20])
 #   q = second_grouping_l['FAMarketOrientation'].quantile(0.25*(j+1))
  #  print(q)
  #  second_grouping_l['MAQuartile']=np.where(second_grouping_l['FAMarketOrientation']>q, second_grouping_l['MAQuartile']+1, second_grouping_l['MAQuartile'])
    


In [ ]:
MAQuartile_grouping_DRC = quartile(second_grouping_DRC, 'FAMarketOrientation',name = 'MAQuartile') 
MAQuartile_grouping_rest = quartile(second_grouping_rest, 'FAMarketOrientation',name = 'MAQuartile') 

In [ ]:
def crop_fac_vill_quart(data, quartilecol, save=None, alpha=.6):
    fig=plt.figure(figsize=(12,12))
    colours = ('red','orange','green','blue')
    for i in range(3):
        axs=fig.add_axes([.07,0.667-0.333*i+.05,.63,0.22])
        for j in range(4):
            grouping = data[data[quartilecol]==j]
            axs.scatter(x=grouping['CropDiv'], y=grouping[grouping.columns[1+i]], alpha = alpha, color=colours[j])
            axs.set_ylabel(str(grouping.columns[1+i]))
            axs.set_xlabel('Average crop diversity in village')
            rp = pearsonr(grouping['CropDiv'], grouping[grouping.columns[1+i]])
            r = round(rp[0],3)
            p = round(rp[1],5)
            n = len(grouping)
            (a,s,Rsq) = ols_params(grouping['CropDiv'], grouping[grouping.columns[1+i]])
            fig.text(0.72, .89-0.33*i-0.05*j,quartilecol+' '+str(j+1)+': n = '+str(n)+'\n   r = '+str(r)+'; p = '+str(p)+'\n   reg = '
                         +str(round(a,3))+'; se = '+str(round(s,3))+
                         '; Rsq = '+str(round(Rsq,3)), color=colours[j])
    if save:
        if analysis == 'PCA':
            fig.savefig('4b PCA '+save)
        else:
            fig.savefig('4b PAF '+save)

In [ ]:
crop_fac_vill_quart(MAQuartile_grouping_DRC, 'MAQuartile', save='crop by village by MA DRC')

In [ ]:
crop_fac_vill_quart(MAQuartile_grouping_rest, 'MAQuartile', save='crop by village by MA rest')

In [ ]:
MAQuartile_DRC = quartile(prep_DRC, 'FAMarketOrientation',name = 'MAQuartile') 
MAQuartile_rest = quartile(prep_rest, 'FAMarketOrientation',name = 'MAQuartile') 

In [ ]:
def crop_fac_quarts_grid(data, quartilecol, save=None, alpha=.07):
    fig = plt.figure(figsize = (16,12))
    colours = ('red','orange','green','blue')
    for i in range(3):
        for j in range(4):
            axes=fig.add_axes([.07+0.24*j,0.667-0.333*i+.05,.19,0.22])
            grouping = data[data[quartilecol]==j]
            axes.scatter(x=grouping['CropDiv'], y=grouping[factors[i]], alpha = alpha, color=colours[j])
            axes.set_ylabel(str(factors[i]))
            axes.set_xlabel('Crop diversity')
            rp = pearsonr(grouping['CropDiv'], grouping[factors[i]])
            r = round(rp[0],3)
            p = round(rp[1],5)
            n = len(grouping)
            (a,s,Rsq)=ols_params(grouping['CropDiv'], grouping[factors[i]])
            axes.set_title('MA Quartile '+str(j+1)+': reg = '
                         +str(round(a,3))+'; se = '+str(round(s,3))+
                         ';\n Rsq = '+str(round(Rsq,3))+' (n = '+str(n)+')'+'\n   r = '+str(r)+'; p = '+str(p), color=colours[j], size=10)
    if save:
        if analysis == 'PCA':
            fig.savefig('4b PCA '+save)
        else:
            fig.savefig('4b PAF ' +save)

In [ ]:
crop_fac_quarts_grid(MAQuartile_DRC, 'MAQuartile', alpha=.35, save = 'MA household grid DRC')

In [ ]:
crop_fac_quarts_grid(MAQuartile_rest, 'MAQuartile', save = 'MA household grid without DRC')

In [ ]:
zero_grouping_DRC = indic_DRC[['CropDiv','HFIAS_code','NrofMonthsFoodInsecure','score_HDDSBadSeason','score_HDDSGoodSeason', 'Coun Reg Vill']].groupby('Coun Reg Vill').mean()
zero_grouping_rest = indic_rest[['CropDiv','HFIAS_code','NrofMonthsFoodInsecure','score_HDDSBadSeason','score_HDDSGoodSeason', 'Coun Reg Vill']].groupby('Coun Reg Vill').mean()

In [ ]:
def grid_graphs(x, y_list, dataframe, save=None, size = 3, graph = 'scatter', alpha = .1, title=None):
    y=len(y_list)
    fig = plt.figure(figsize = (size*3,size*y))
    ax=[]
    for j in range(y):
        data = dataframe[[x,y_list[j]]].dropna()
        ax.append(fig.add_axes([0.1, 1-np.divide(j,y)-0.75*np.divide(1,y), 0.5, 0.75*np.divide(1,y)]))
        if graph=='hexbin':
            ax[j].hexbin(data[x],data[y_list[j]], gridsize=int(0.5*max(data[x])), cmap='Greens')
        elif graph=='scatter':
            ncirc = max(np.divide(len(data[x].unique()),1.8), np.divide(len(data[y_list[j]].unique()),0.8))
            if ncirc < 25:
                ax[j].scatter(data[x],data[y_list[j]], alpha = alpha, s=np.divide(12000,ncirc*ncirc))
            else:
                ax[j].scatter(data[x],data[y_list[j]], alpha = alpha)
        else:
            print('must be hexbin or scatter')
            return()
        ax[j].set_ylabel(y_list[j])
        ax[j].set_xlabel(x)
        if title:
            ax[j].set_title(title)
        rp = pearsonr(data[x], data[y_list[j]])
        r = round(rp[0], 3)
        p=round(rp[1],5)
        (a, s, Rsq) = ols_params(data[x], data[y_list[j]])
        (A, S, RSQ) = ols_params(data[x], norm_series(data[y_list[j]]))
        fig.text(0.6*1.01,1-0.9*np.divide(1,y)-np.divide(j,y)+0.8*np.divide(1,y)*0.8,'Corr coef: '+str(r)+' Sig level: '+str(p)+
                   ' (n = '+str(len(data))+')'+'\nReg coeff: '+str(round(a,3))+' Std err: '+str(round(s,3))+' Rsq: '+
                   str(round(Rsq,3))+'\nNormalised Reg: '+str(round(A,3))+' Std err: '+str(round(S,3)))    
    if save:
        fig.savefig(save)

In [ ]:
grid_graphs('CropDiv', ['HFIAS_code','NrofMonthsFoodInsecure','score_HDDSBadSeason','score_HDDSGoodSeason'],indic_DRC, 
            graph = 'scatter', alpha=.1, save = '4b crop diversity by indicators houdehold level DRC',
           title = 'All available households in DRC')

In [ ]:
grid_graphs('CropDiv', ['HFIAS_code','NrofMonthsFoodInsecure','score_HDDSBadSeason','score_HDDSGoodSeason'],indic_rest, 
            graph = 'scatter', alpha=.008, save = '4b crop diversity by indicators houdehold level without DRC',
           title='All available households except DRC')

In [ ]:
grid_graphs('CropDiv', ['HFIAS_code','NrofMonthsFoodInsecure','score_HDDSBadSeason','score_HDDSGoodSeason'],zero_grouping_DRC, 
            graph = 'scatter', save = '4b crop diversity by indicators village level DRC', alpha=.35,
           title = 'Grouped by village in DRC')

In [ ]:
grid_graphs('CropDiv', ['HFIAS_code','NrofMonthsFoodInsecure','score_HDDSBadSeason','score_HDDSGoodSeason'],zero_grouping_rest, 
            graph = 'scatter',save = '4b crop diversity by indicators village level without DRC', alpha=.2,
           title='Grouped by village without DRC')